## Importação das Bibliotecas necessárias
A biblioteca copy foi usada para realizar uma cópia de valores ao longo do algoritmo de pré-processamento

In [ ]:
import pandas as pd
import numpy as np
import operator
from copy import deepcopy

## Leitura dos Arquivos

In [ ]:
music_data = pd.read_csv("music_data.csv")
hits = pd.read_csv("hits.csv")
target = pd.read_csv("target.csv")

## Inicialização das variáveis de controle
Estas variáveis serão usadas ao longo do código para facilitar a legibilidade

In [ ]:
music_id = 0
music = 1
music_value = 4
usr_id = 0
usr_musics = 1
topHits = 100  # decidi por utilizar 100 top hits para a recomendação
numUsers = len(hits.groupby(['user_id']).count())-1

# Algoritmo de Pré-Processamento

É necessário um algoritmo de pré processamento para deixar os dados do jeito correto estabelecido para o funcionamento do algoritmo.
Decidi usar a seguinte abordagem: Terei uma matriz com todos os usuários. Cada usuário será uma linha da matriz. Esta matriz (chamada users) tem duas colunas: a primeira para o id do usuário naquela linha, e a segunda que é outra matriz (chamada listaMusicas). Esta contém, na primeira coluna, os 100 hits mais tocados, e na segunda coluna, o quanto aquele determinado usuário ouviu àquela música.
O algoritmo de pré-processamento não se preocupa com o caso de usuários novos que ainda não ouviram a nenhuma música, pois o objetivo era recomendar apenas para usuários target já pré-definidos que já haviam ouvido alguma música.


## Inicialização das matrizes usadas pelo algoritmo de pré-processamento

In [ ]:
musics = [[]for i in range(topHits)]
# modelo do vetor user = [0,[[105301,69],[51,0],...]]
users = [[]for i in range(numUsers)]

# pegando as top 100 musicas, baseado em numero de plays
music_data.sort_values(['plays'], ascending=False, inplace=True)
top_musics = np.array(music_data['music_id'][:topHits])

# fazendo o vetor base de musicas com todos os valores 0
for i in range(len(top_musics)):
    musics[i].append(top_musics[i])
    musics[i].append(0)

## Atribuição dos valores ouvidos de cada top_hit por cada usuário
A matriz de cada usuário possui vários valores 0, pois dificilmente um usuário terá escutado todas as músicas do top 100. Como o vetor base deixa como padrão o valor 0 para todas as músicas, a parte abaixo é encarregada de sobrescrever estes valores 0 nas músicas que o usuário já ouviu, com o valor correto, e fazer isto para todos os usuários da base de dados.

In [ ]:
linha = 0
for j in range(numUsers):
    users[j] = [hits.iloc[linha, usr_id], musics]  # users[j] = [user_id, vetor base com os top 100 hits]
    if linha < len(hits):  # if usado para garantir que na última iteração não ocorra um index out of bounds
        #resetando listaMusicas para que cada usuário tenha um vetor limpo inicialmente:
        listaMusicas = deepcopy(musics)  # deepcopy para que apenas os valores de musics sejam transferidos para listaMusicas. Particularidade da linguagem Python.
        while hits.iloc[linha, usr_id] == j:  # hits tem várias linhas de informação para cada usuário. Enquanto eu estiver no mesmo usuário faça:
            musica = hits.iloc[linha, music]
            for i in range(len(top_musics)):
                if users[j][usr_musics][i][music_id] == musica:  # se a musica atual em hits existir no vetor do usuario(tambem vale: se estiver no top 100)
                    listaMusicas[i][music] = hits.iloc[linha][music_value]
            linha += 1
        users[j] = [hits.iloc[linha, usr_id], listaMusicas]  # atribui listaMusicas com os valores atualizados daquele usuário à sua matriz

# Algoritmo K-Nearest
Decidi por utilizar um algoritmo do tipo KNN para definir as recomendações. Utilizei uma função para calcular as distâncias euclidianas entre os vetores. Ordenei estas distâncias, separei usuários parecidos de acordo com o valor de k e, em um vetor (chamado vetComb) somei todos os valores de todos os top 100 hits destes usuários (inclusive os coincidentes, uma vez que se uma música foi MUITO ouvida por usuários parecidos com o target, esta provavelmente é uma boa recomendação). Então, ordenei de forma decrescente este vetComb (o nome vem de "vetor combinação") de forma que, nas primeiras n posições eu tivesse as n melhores recomendações. O retorno final da função é justamente este vetor, fatiado para conter apenas os id's das músicas recomendadas, excluíndo o valor da distância vetorial calculada, já que este valor passou a ser irrelevante.

## Função para calcular a distância
Faz uso da biblioteca numpy como np para calcular a disância euclidiana entre o user target e um vetor de usuário qualquer da base de dados.

In [ ]:
def distance(user_target, one_user, length):
    dist = 0
    for i in range(length):
        dist += np.square(user_target[1][i][1] - one_user[1][i][1])
    return np.sqrt(dist)

## Função K-nearest

### Pseudo-código do trecho do algoritmo knn:
    calcular distâncias euclidianas e guarda em um vetor distances
    ordenar distances
    vetor combinado = musics, para que seja uma matriz com os top hits e valor base 0 que será alterado à medida que encontrar valores para aquela música
    para cada usuario dentro de k em sortedDistances
        pega o user correspondente no vetor users
            para cada musica que a pessoa escutou
                soma o valor dessa musica na posicao dela no vetor combinado
    # vetor combinado = [[music_id1,music_value1],[music_id2,music_value2],[music_id3,music_value3]...]
    # recomendacoes = vetor combinado - user_target => farei isso com um for:
    recomendacoes = [[]for i in range(vetComb)]
    for i in range(len(vetor combinado))   
        recomendacoes[i].append(vetComb[i][0])
        #a linha abaixo significa: vetComb.music_value[1] - user.music_value[1]...
        recomendacoes[i].append(vetor combinado[i][1] - user_target[1][i][1])
    ordenar o vetor recomendacoes em ordem decrescente pelas valores das musicas
    # porque se o usr_target escutou uma musica 0 tempo e os parecidos com ele escutaram 10000000 tempo (por exemplo),
    # essa recomendacao tem que estar lá no topo.
    retorna recomendacoes
    #na funcao get_recomendations, eu fatiarei o vetor recomendacoes no numero n de recomendacoes que for passado passado como parametro

In [ ]:
def knearest(all_users, user_target, k):
    distances = {}
    length = len(user_target[1])  # user_target[1] se refere à segunda coluna de user, que contém os top hits

    for i in range(len(all_users)):
        dist = distance(user_target, all_users[i], length)
        distances[i] = dist

    # sortedDistances tem, de forma ordenada crescente, os usuarios com o gosto mais parecido com o que eu passei.
    # na posicao 0 eu tenho o usuario mais parecido com o que eu passei.
    sortedDistances = sorted(distances.items(), key=operator.itemgetter(1))

    usuariosParecidos = [[]for i in range(k)]

    for i in range(k):
        usuariosParecidos[i] += sortedDistances[i]
    # aqui, em usuariosParecidos, eu tenho o id dos usuarios mais parecidos e a distancia vetorial entre cada um deles e user target

    vetComb = musics

    for i in range(len(usuariosParecidos)):
        usrTemp = users[usuariosParecidos[i][usr_id]]
        for ind in range(len(usrTemp[usr_musics])): # quantidade de musicas que tem no top do usrTemp
            vetComb[ind][1] += usrTemp[usr_musics][ind][music_value]

    recomendacoes = [[]for i in range(len(vetComb))]

    for i in range(len(vetComb)):
        recomendacoes[i].append(vetComb[i][0])  # recomendacoes[i][0] tem o id da musica
        recomendacoes[i].append(vetComb[i][1] - user_target[1][i][1])  # recomendacoes[i][1] tem o valor subtraido do quanto os usuarios parecidos escutaram aquela musica com o quanto o user_target a ouviu

    sortedRecomendacoes = sorted(recomendacoes, key=operator.itemgetter(1), reverse=True)
    finalRecomendations = []

    for i in range(len(sortedRecomendacoes)):
        finalRecomendations.append(sortedRecomendacoes[i][0])

    return finalRecomendations

## getRecommendations
A função getRecommendations tem a responsabilidade de definir os usuários target a partir da tabela "target.csv" e passá-los, um a um, para a função knearest, e por último, inserir cada resposta de recomendação em um dict que será retornado

In [1]:
def getRecommendations(n):
    user_target = users[0]
    recommended = knearest(users, user_target, 5)[:n]

    recommendations = {
        "user_id": user_target[0],
        "recommendations": recommended
    }

    return recommendations